In [ ]:
!pip install pandasql gower kmodes category_encoders -q

In [ ]:
import pandas as pd
import numpy as np
from pandasql import sqldf

import math
from scipy import stats
from scipy.stats import geom
from sklearn.linear_model import LinearRegression
from statsmodels.stats.power import TTestIndPower, ttest_power

import re
from tqdm.notebook import tqdm

import seaborn as sns
from plotly import tools
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as py
import matplotlib.pyplot as plt
%matplotlib inline

# import gower
# import lightgbm as lgb
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# from category_encoders import TargetEncoder
# from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
# from sklearn.metrics import r2_score, mean_squared_error, roc_auc_score, classification_report, mean_absolute_error
# from keras.callbacks import ModelCheckpoint
# from keras.models import Sequential
# from keras.layers import Dense, Activation, Flatten
# from kmodes.kprototypes import KPrototypes
# from sklearn.cluster import KMeans, DBSCAN
# from mpl_toolkits.mplot3d import Axes3D
# from yellowbrick.classifier import ROCAUC

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category = DeprecationWarning)

R_STATE = 18714836 # random state

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

## Data Loading and Cleaning

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Capstone/fps_videogames.csv')

In [ ]:
rename = {'CpuName': 'CPU Name', 'CpuNumberOfCores': 'CPU Cores', 'CpuNumberOfThreads': 'CPU Threads',
          'CpuBaseClock': 'CPU Base Clock', 'CpuCacheL1': 'CPU Cache L1', 'CpuCacheL2': 'CPU Cache L2',
          'CpuCacheL3': 'CPU Cache L3', 'CpuDieSize': 'CPU Die Size', 'CpuFrequency': 'CPU Frequency',
          'CpuMultiplier': 'CPU Multiplier', 'CpuMultiplierUnlocked': 'CPU Multiplier Unlocked',
          'CpuProcessSize': 'CPU Process Size', 'CpuTDP': 'CPU TDP', 'CpuNumberOfTransistors': 'CPU Transistors',
          'CpuTurboClock': 'CPU Turbo Clock',
          'GpuName': 'GPU Name', 'GpuArchitecture': 'GPU Architecture', 'GpuBandwidth': 'GPU Bandwidth',
          'GpuBaseClock': 'GPU Base Clock', 'GpuBoostClock': 'GPU Boost Clock', '\'GpuBus': 'GPU Bus',
          'GpuNumberOfComputeUnits': 'GPU Compute Units', 'GpuDieSize': 'GPU Die Size', 'GpuDirectX': 'GPU Direct X',
          'GpuNumberOfExecutionUnits': 'GPU Execution Units', 'GpuFP32Performance': 'GPU FP32 Performance',
          'GpuMemoryBus': 'GPU Memory Bus', 'GpuMemorySize': 'GPU Memory Size', 'GpuMemoryType': 'GPU Memory Type',
          'GpuOpenCL': 'GPU Open CL', 'GpuOpenGL': 'GPU Open GL', 'GpuPixelRate': 'GPU Pixel Rate', 'GpuProcessSize': 'GPU Process Size',
          'GpuNumberOfROPs': 'GPU Number of ROPs', 'GpuShaderModel': 'GPU Shader Model', 'GpuNumberOfShadingUnits': 'GPU Shading Units',
          'GpuNumberOfTMUs': 'GPU TMUs', 'GpuTextureRate': 'GPU Texture Rate', 'GpuNumberOfTransistors': 'GPU Transistors', 'GpuVulkan': 'GPU Vulkan',
          'GameName': 'Game', 'GameResolution': 'Game Resolution', 'GameSetting': 'Game Settings'}

df.rename(columns=rename, inplace=True)

df['CPU Name'] = df['CPU Name'].apply(lambda x: x.replace('-', ' '))
df['CPU Brand'] = df['CPU Name'].apply(lambda x: x.split(' ')[0])

df['CPU Type'] = df['CPU Name'].apply(lambda x: re.findall('[a-zA-Z]+', x)[-1].upper() if x[-1].isalpha() else 'Normal')
df['CPU Name'] = df['CPU Name'].apply(lambda x: x.replace(re.findall('[a-zA-Z]+', x)[-1], '', -1).strip() if x[-1].isalpha() else x)

df['CPU Model'] = df['CPU Name']
df['CPU Model'] = df.apply(lambda x : x['CPU Model'].replace(str(x['CPU Brand']), '').strip(), axis=1)

cpu_series = ['A4', 'A6', 'Athlon', 'Athlon 64', 'Athlon II', 'FX', 'Ryzen', 'Core', 'Pentium']

for series in cpu_series:
    df['CPU Series Temp'] = df['CPU Model'].apply(lambda x: series if series in x else 'NA')
    ind = df[df['CPU Series Temp']==series].index
    df.loc[ind, 'CPU Series'] = df.loc[ind, 'CPU Series Temp']

df['CPU Model'] = df.apply(lambda x : x['CPU Model'].replace(str(x['CPU Series']), '').strip(), axis=1)
df.drop(['CPU Series Temp'], axis=1, inplace=True)

df['CPU Generation'] = df['CPU Model'].apply(lambda x: x.split(' ')[-1][1] if x.split(' ')[-1][0].isalpha() else x.split(' ')[-1][0]).tolist()

core_ind = df[df['CPU Series']=='Core'].index

df.loc[core_ind, 'CPU Generation'] = df.loc[core_ind, 'CPU Model'].apply(lambda x: x.split(' ')[-1][0]+'0' if len(x.split(' ')[-1])==3 else x.split(' ')[-1][0])
older_gen = {'90': 1, '80': 0, '70': -1, '60': -2, '50': -3, '40': -4, '(': None}
df['CPU Generation'] = df['CPU Generation'].replace(older_gen)

df['CPU Model'] = df.apply(lambda x : x['CPU Model'].replace(str(x['CPU Model'].split(' ')[-1]), '').strip(), axis=1)

df['CPU Model'] = df['CPU Model'].fillna('NA')
df['CPU Series'] = df['CPU Series'].fillna('NA')
df.replace({'?': None}, inplace=True)

df['GPU Transistors'] = df['GPU Transistors'].astype(float)
df['CPU Transistors'] = df['CPU Transistors'].astype(float)

df['GPU Die Size'] = df['GPU Die Size'].astype(float)
df['CPU Die Size'] = df['CPU Die Size'].astype(float)

In [ ]:
cpu_cols = ['CPU Name', 'CPU Brand', 'CPU Model', 'CPU Series', 'CPU Generation', 'CPU Type', 'CPU Cores', 'CPU Threads',
            'CPU Base Clock', 'CPU Cache L1', 'CPU Cache L2', 'CPU Cache L3', 'CPU Die Size','CPU Frequency', 'CPU Multiplier',
            'CPU Multiplier Unlocked', 'CPU Process Size', 'CPU TDP', 'CPU Transistors', 'CPU Turbo Clock']

cpu_detail_cols = ['CPU Brand', 'CPU Model', 'CPU Series', 'CPU Generation']

gpu_cols = ['GPU Name', 'GPU Architecture', 'GPU Bandwidth', 'GPU Base Clock', 'GPU Boost Clock', 'GPU Bus', 'GPU Compute Units',
            'GPU Die Size', 'GPU Direct X', 'GPU Execution Units', 'GPU FP32 Performance', 'GPU Memory Bus', 'GPU Memory Size',
            'GPU Memory Type', 'GPU Open CL', 'GPU Open GL', 'GPU Pixel Rate', 'GPU Process Size', 'GPU Number of ROPs', 'GPU Shader Model',
            'GPU Shading Units', 'GPU TMUs', 'GPU Texture Rate', 'GPU Transistors', 'GPU Vulkan']

gpu_detail_cols = ['GPU Brand', 'GPU Model', 'GPU Series', 'GPU Generation']


cpu = df[cpu_cols]
gpu = df[gpu_cols]

In [ ]:
# gpu['GPU Name Temp'] = gpu['GPU Name']
# gpu['GPU Brand'] = gpu['GPU Name Temp'].apply(lambda x: x.split(' ')[0])
# gpu['GPU Name Temp'] = gpu['GPU Name Temp'].apply(lambda x: ' '.join(x.split(' ')[1:]))

# gpu['GPU Model'] = gpu['GPU Name Temp'].apply(lambda x: x.split(' ')[0])
# gpu['GPU Name Temp'] = gpu['GPU Name Temp'].apply(lambda x: ' '.join(x.split(' ')[1:]))

# gpu['GPU Series'] = gpu['GPU Name Temp'].apply(lambda x: x.split(' ')[0])
# gpu['GPU Name Temp'] = gpu['GPU Name Temp'].apply(lambda x: ' '.join(x.split(' ')[1:]))

# gpu['GPU Generation'] = gpu['GPU Name Temp'].apply(lambda x: x.split(' ')[0])
# gpu['GPU Name Temp'] = gpu['GPU Name Temp'].apply(lambda x: ' '.join(x.split(' ')[1:]))

In [ ]:
cpu_null_cols = ['CPU Cache L3', 'CPU Die Size', 'CPU Transistors']
gpu_null_cols = ['GPU Bandwidth', 'GPU Compute Units', 'GPU Compute Units', 'GPU Compute Units', 'GPU Die Size',
                 'GPU Execution Units', 'GPU FP32 Performance', 'GPU Memory Bus', 'GPU Memory Size', 'GPU Memory Type',
                 'GPU Open CL', 'GPU Shader Model', 'GPU Shading Units', 'GPU Transistors', 'GPU Vulkan']

## Exploratory Data Analysis

In [ ]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0: 'Missing Values', 1: '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
    print(f'Your selected dataframe has {str(df.shape[1])} columns.\nThere are {str(mis_val_table_ren_columns.shape[0])} columns that have missing values.')
    return mis_val_table_ren_columns

In [ ]:
df.head()

In [ ]:
missing_values_table(df)

In [ ]:
gpu1 = df[['GPU Process Size', 'GPU Transistors', 'GPU Die Size']]
gpu1.dropna(inplace=True)
cpu1 = df[['CPU Process Size', 'CPU Transistors', 'CPU Die Size']]
cpu1.dropna(inplace=True)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=cpu1['CPU Process Size'], y=cpu1['CPU Transistors'], z=cpu1['CPU Die Size'],
    mode='markers',
    marker=dict(
        size=cpu1['CPU Transistors'].apply(lambda x: x*0.006),
        color=cpu1['CPU Process Size'],
        colorscale='Viridis',
        opacity=0.8
    )
)])

fig.update_layout(title='(CPU) Process Size vs Die Size vs Transistors',
                  autosize=False,
                  width=800,
                  height=800,
                  scene = dict(
                      xaxis_title='Process Size',
                      yaxis_title='Transistors',
                      zaxis_title='Die Size'))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=gpu1['GPU Process Size'], y=gpu1['GPU Transistors'], z=gpu1['GPU Die Size'],
    mode='markers',
    marker=dict(
        size=gpu1['GPU Transistors'].apply(lambda x: x*0.003),
        color=gpu1['GPU Process Size'],
        colorscale='Viridis',
        opacity=0.8
    )
)])

fig.update_layout(title='(GPU) Process Size vs Die Size vs Transistors',
                  autosize=False,
                  width=800,
                  height=800,
                  scene = dict(
                      xaxis_title='Process Size',
                      yaxis_title='Transistors',
                      zaxis_title='Die Size'))
fig.show('svg')

In [ ]:
cpu2 = df[['CPU Brand', 'CPU Series', 'CPU Model', 'CPU Generation', 'CPU Type']]
cpu2['Count'] = 1

fig = px.sunburst(cpu2, path=['CPU Brand', 'CPU Series', 'CPU Model'], values='Count', title='Pie Chart of CPUs in our dataset',
            color='CPU Brand', color_continuous_scale='Viridis', color_continuous_midpoint=5)
fig.show()

In [ ]:
game = df[['Game', 'Game Resolution', 'Game Settings']]
game['Count'] = 1

game_count = game.groupby(['Game']).agg('count').reset_index().drop_duplicates()
fig = px.bar(game_count, x='Game', y='Count')
fig.show()

In [ ]:
game = df[['Game', 'Game Resolution', 'Game Settings']]
game['Count'] = 1

game_reso = game.groupby(['Game Resolution']).agg('count').reset_index().drop_duplicates()
fig = px.bar(game_reso, x='Game Resolution', y='Count')
fig.show()

In [ ]:
game = df[['Game', 'Game Resolution', 'Game Settings']]
game['Count'] = 1

game_sett = game.groupby(['Game Settings']).agg('count').reset_index().drop_duplicates()
fig = px.bar(game_sett, x='Game Settings', y='Count')
fig.show()

In [ ]:
corr = df.corr()

fig = go.Figure(data=go.Heatmap(z=corr, x=corr.columns, y=corr.columns,
                                xgap=1, ygap=1, colorscale='Viridis'),
                layout=go.Layout(title_text='Correlation Plot', height=1000,
                                 yaxis_autorange='reversed'))

fig.show()

## Inference

In [ ]:
cpu_years = pd.read_excel('/content/gdrive/MyDrive/Capstone/cpu_names.xlsx')

In [ ]:
cpu_detail_cols = ['CPU Brand', 'CPU Model', 'CPU Series', 'CPU Generation', 'CPU Cores']

cpu_years['CPU Name'] = cpu_years['CPU Name'].apply(lambda x:x.replace('-', ' '))
# cpu_years.drop(['month'], axis=1, inplace=True)
cpu = pd.merge(df[cpu_detail_cols + ['CPU Name', 'CPU Transistors']], cpu_years, how='left', on='CPU Name')
cpu = cpu[~cpu['CPU Transistors'].isna() & ~cpu['CPU Generation'].isna()] #dropping na transistors

cpu.drop(['Unnamed: 3'], axis=1, inplace=True)
numeric_cols = ['CPU Transistors', 'CPU Generation']
cpu[numeric_cols] = cpu[numeric_cols].apply(pd.to_numeric)

cpu.dropna(inplace = True)
cpu = cpu.drop_duplicates()
cpu['Adjusted Transistors'] = cpu['CPU Transistors']/cpu['CPU Cores']
cpu['Adjusted Transistors'] = cpu['Adjusted Transistors'].apply(int)

cpu.sort_values('CPU Release Year')

temp = cpu.copy()
new_cols = [x.replace(' ', '_') for x in cpu.columns]
print(new_cols)
temp.columns = new_cols
temp.head()

In [ ]:
out = sqldf('''

    select max(Adjusted_Transistors) as Max_transistors, count(Adjusted_Transistors) as Models_released,
    avg(Adjusted_Transistors) as Avg_transistors,
    case when month < 1 then CPU_Release_Year - 1 
    when month > 9 then CPU_Release_Year + 1
    else CPU_Release_Year
    end as new_year
    from temp group by new_year

''')

In [ ]:
out['Double_Transistors'] = 2 * out['Max_transistors']
out['Actual_Transistors'] = out['Max_transistors'].shift(-2)

In [ ]:
out = out.replace(out.iloc[13][0], 1200) 
out = out.replace(out.iloc[6][0], 1303)
out = out.replace(out.iloc[7][0], 1303)
out = out.replace(out.iloc[11][0], 3100)

In [ ]:
out

In [ ]:
stat1, p1 = stats.ks_2samp(out['Actual_Transistors'][:-2], out['Double_Transistors'][:-2])

In [ ]:
stat2, p2 = stats.mannwhitneyu(out['Actual_Transistors'][:-2], out['Double_Transistors'][:-2])

In [ ]:
n1 = 12
n2 = 12
mu = (n1*n2)/2
sigma = math.sqrt((n1*n2)*(n1+n2+1)/12)
z = (stat2 - mu)/sigma
r = abs(z)/math.sqrt(n1)

In [ ]:
r

In [ ]:
kde_df = out[['Actual_Transistors', 'Double_Transistors']]

In [ ]:
ax = kde_df.plot.kde()
plt.xlabel('Transistors')
plt.ylabel('Density')
plt.title('Comparison of Distribution Between Actual Transistors and Double Transistors')
plt.savefig('KDE Plot Of Distributions')

In [ ]:
one = out['Double_Transistors'][:-2]
two = out['Actual_Transistors'][:-2]

In [ ]:
X = out['new_year'][:-2]
X_axis = np.arange(len(X))
plt.bar(X_axis - 0.2, one, 0.4, label = 'Double_Transistors')
plt.bar(X_axis + 0.2, two, 0.4, label = 'Actual_Transistors')
plt.xticks(X_axis, X)
plt.xlabel('Year')
plt.ylabel('Number of Transistors')
plt.title('Comparison of Distribution Between Actual Transistors and Double Transistors')
plt.xticks(rotation = 45)
plt.legend()
plt.show()
plt.savefig('Bar Plot Of Distributions')

In [ ]:
errorBar_df = out.copy()

In [ ]:
errorBar_df['Errors'] = abs(errorBar_df['Double_Transistors'] - errorBar_df['Actual_Transistors'])

In [ ]:
errorBar_df

In [ ]:
sem_Actual = out['Actual_Transistors'].std() / math.sqrt(12)
sem_Actual

In [ ]:
sem_Expected = out['Double_Transistors'].std() / math.sqrt(12)
sem_Expected

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))
ax.bar(errorBar_df['new_year'], errorBar_df['Actual_Transistors'],
       yerr=sem_Actual,
       align='center',
       alpha=0.5,
       ecolor='black',
       capsize=10)
plt.xlabel('Year')
plt.ylabel('Actual Number of Transistors')

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))
ax.bar(errorBar_df['new_year'], errorBar_df['Double_Transistors'],
       yerr=sem_Expected,
       align='center',
       alpha=0.5,
       ecolor='black',
       capsize=10)
plt.xlabel('Year')
plt.ylabel('Expected Number of Transistors')

## Data Imputation

In [ ]:
based_col = ['CPU Brand', 'CPU Model', 'CPU Series', 'CPU Generation']

geometric_cols = ['CPU Transistors']
linear_cols = ['CPU Die Size']
step_cols = []

# Geometric Imputation
if geometric_cols:
    for impute_col in geometric_cols:
        temp = df[based_col + [impute_col]].dropna(inplace=False)
        temp[impute_col] = pd.to_numeric(temp[impute_col])
        index = df[df[impute_col].isnull()].index.tolist()

        value = temp.groupby(based_col).agg('mean').reset_index().to_dict('split')['data']
        value_keys = [val[:4] for val in value]
        for ind in tqdm(index):

            key = tuple(df.loc[ind, based_col].values)
            previous = [val for val in value if key[:3] == tuple(val[:3])]
            if not previous:
                df.drop(ind, axis=0, inplace=True)
                continue
            gen = [int(p[3]) for p in previous]
            impute = [int(p[-1]) for p in previous]    

            if list(key) in value_keys:
                df.loc[ind, impute_col] = [val for val in value if key[:4] == tuple(val[:4])][0][-1]

            else:
                if int(key[3]) > min(gen) and int(key[3]) < max(gen):
                    geom = np.geomspace(impute[gen.index(min(gen))],
                                        impute[gen.index(max(gen))],
                                        max(gen) - min(gen) + 1)
                    df.loc[ind, impute_col] = geom[int(key[-1])-min(gen)]

                if int(key[3]) < min(gen):
                    gap = min(gen) - int(key[3])
                    val = impute[gen.index(min(gen))]
                    for i in range(gap): val /= 2
                    df.loc[ind, impute_col] = val
                
                else:
                    gap = int(key[3]) - max(gen)
                    val = impute[gen.index(min(gen))]
                    for i in range(gap): val *= 2
                    df.loc[ind, impute_col] = val


# Linear Imputation
if linear_cols:
    for impute_col in linear_cols:
        temp = df[based_col + [impute_col]].dropna(inplace=False)
        temp[impute_col] = pd.to_numeric(temp[impute_col])
        index = df[df[impute_col].isnull()].index.tolist()

        value = temp.groupby(based_col).agg('mean').reset_index().to_dict('split')['data']
        value_keys = [val[:4] for val in value]
        for ind in tqdm(index):

            key = tuple(df.loc[ind, based_col].values)
            previous = [val for val in value if key[:3] == tuple(val[:3])]
            if not previous:
                df.drop(ind, axis=0, inplace=True)
                continue
            gen = [int(p[3]) for p in previous]
            impute = [int(p[-1]) for p in previous]    

            if list(key) in value_keys:
                df.loc[ind, impute_col] = [val for val in value if key[:4] == tuple(val[:4])][0][-1]

            else:
                lr = LinearRegression()
                lr.fit(np.array(gen).reshape(-1, 1), np.array(impute).reshape(-1, 1))
                df.loc[ind, impute_col] = lr.predict([[7]])[0][0]


# Step Imputation
if step_cols:
    for impute_col in step_cols:
        temp = df[based_col + [impute_col]].dropna(inplace=False)
        temp[impute_col] = pd.to_numeric(temp[impute_col])
        index = df[df[impute_col].isnull()].index.tolist()

        value = temp.groupby(based_col).agg('mean').reset_index().to_dict('split')['data']
        value_keys = [val[:4] for val in value]
        for ind in tqdm(index):

            key = tuple(df.loc[ind, based_col].values)
            previous = [val for val in value if key[:3] == tuple(val[:3])]
            if not previous:
                df.drop(ind, axis=0, inplace=True)
                continue
            gen = [int(p[3]) for p in previous]
            impute = [int(p[-1]) for p in previous]    

            if list(key) in value_keys:
                df.loc[ind, impute_col] = [val for val in value if key[:4] == tuple(val[:4])][0][-1]

            else:
                steps = np.linspace(impute[gen.index(min(gen))],
                                    impute[gen.index(max(gen))],
                                    max(gen) - min(gen) + 1)
                step = step[1] - step[0]
                if int(key[3]) > min(gen) and int(key[3]) < max(gen):
                    df.loc[ind, impute_col] = steps[int(key[-1])-min(gen)]

                if int(key[3]) < min(gen):
                    gap = min(gen) - int(key[3])
                    val = impute[gen.index(min(gen))]
                    for i in range(gap): val -= step
                    df.loc[ind, impute_col] = val
                
                else:
                    gap = int(key[3]) - max(gen)
                    val = impute[gen.index(min(gen))]
                    for i in range(gap): val += step
                    df.loc[ind, impute_col] = val

In [ ]:
 missing_values_table(df)